# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import pandas as pd
import numpy as np
from IPython.display import display
import string                          #library of string constants
import re
from sqlalchemy import create_engine
import nltk                            # natural language tool kit

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# load data from database
engine = create_engine('sqlite:///DR.db')
df = pd.read_sql_table('messages', engine)
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis=1)
cols=list(Y.columns)


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    Description: This function reads a text string
                    removes punctuation
                    converts to lower case
                    strips out white space
                    tokenizes the text into a word list
                    Stems the word list
                    removes stop words

    Arguments: Initial test string
    Returns: clean list of word tokens
    """

    stop_words = nltk.corpus.stopwords.words("english")

    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip())

    #tokenize
    words = nltk.word_tokenize (text)

    #stem
    #A processing interface for removing morphological affixes from words. This process is known as stemming.
    # (https://www.nltk.org/api/nltk.stem.html)

    clean_tokens = [nltk.PorterStemmer().stem(w) for w in words if w not in stop_words]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
forest_clf = RandomForestClassifier(n_estimators=10)
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(forest_clf))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.33, random_state = 42)

pipeline.fit(X_train.values, Y_train.values)

#pipeline.fit(X_train.as_matrix(), Y_train.as_matrix())
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def report_results(y_test, y_pred):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
    from sklearn.metrics import precision_recall_fscore_support
    
    #display(y_pred,Y_test)
    """
    Description: This function reads in the Y test and the Y predicted values
                    it used the sklearn precision_recall_fscore_support function
                    to compute the fscore, precision and recall values for each catigory
                    it places them in a dataframe for nice formating
                    lastly it compute the average of these score to provide and view
    Arguments: the y test and y predicted values
    Returns: a summary report
    """
    report = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    for indx, cat in enumerate(y_test.columns):
        idx=indx+1
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,indx], average='weighted')
        report.at[idx,'Category']=cat
        report.at[idx,'f_score']=f_score
        report.at[idx,'precision']=precision
        report.at[idx,'recall']=recall

    rd={}
    rd['Precision (averaged)']=report['precision'].mean()
    rd['Recall (averaged)']=report['recall'].mean()
    rd['F_score (averaged)']=report['f_score'].mean()
    display(pd.DataFrame.from_dict(rd,orient='index').rename(columns={0:"Score"}))
    report.set_index('Category', inplace=True)
    return report

#results = get_results(y_prediction_test, y_prediction_train)
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    display(report_results(Y_test, y_pred))

cd={}

from sklearn.metrics import classification_report
for i, cat in enumerate(Y_test):
    print(cat)
    print(classification_report(Y_test[cat], y_pred[:, i]))
  


,Score
Precision (averaged),0.933712
Recall (averaged),0.943892
F_score (averaged),0.931616


,f_score,precision,recall
Category,,,
related,0.788015,0.786358,0.800277
request,0.866448,0.872207,0.881184
offer,0.993417,0.991235,0.995608
aid_related,0.739903,0.741969,0.744105
medical_help,0.89699,0.900328,0.921983
medical_products,0.930422,0.942531,0.950763
search_and_rescue,0.965769,0.969274,0.975266
security,0.972,0.963344,0.980814
military,0.960053,0.963714,0.970874


related
             precision    recall  f1-score   support

          0       0.63      0.44      0.52      2054
          1       0.84      0.92      0.88      6534
          2       0.36      0.42      0.39        64

avg / total       0.79      0.80      0.79      8652

request
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      7180
          1       0.78      0.42      0.55      1472

avg / total       0.87      0.88      0.87      8652

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      8652

aid_related
             precision    recall  f1-score   support

          0       0.76      0.84      0.79      5107
          1       0.72      0.61      0.66      3545

avg / total       0.74      0.74      0.74      8652

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
def improve_model(pipeline,X_train,Y_train):
    
    from sklearn.model_selection import GridSearchCV
    parameters = {'vect__min_df': [5],
                  'tfidf__use_idf':[True, False],
                  'clf__estimator__n_estimators':[25], 
                  'clf__estimator__min_samples_split':[2, 5]}

    #scorer = make_scorer(performance_metric)
    cv = GridSearchCV(pipeline, param_grid = parameters,  verbose = 1)

    # Find best parameters
    np.random.seed(81)
    tuned_model = cv.fit(X_train, Y_train)
    return tuned_model

tuned_model=improve_model(pipeline,X_train,Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 17.9min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_pred = tuned_model.predict(X_test)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    display(report_results(Y_test, y_pred))


from sklearn.metrics import classification_report
for i, cat in enumerate(Y_test):
    print(cat)
    print(classification_report(Y_test[cat], y_pred[:, i]))





,Score
Precision (averaged),0.938160
Recall (averaged),0.948008
F_score (averaged),0.937883


,f_score,precision,recall
Category,,,
related,0.800109,0.801383,0.813222
request,0.883136,0.887543,0.893897
offer,0.993417,0.991235,0.995608
aid_related,0.763028,0.762815,0.763292
medical_help,0.899214,0.901509,0.922446
medical_products,0.936961,0.944881,0.953074
search_and_rescue,0.963366,0.965846,0.974341
security,0.972494,0.967674,0.980929
military,0.959934,0.959871,0.969949


related
             precision    recall  f1-score   support

          0       0.69      0.46      0.55      2054
          1       0.84      0.93      0.88      6534
          2       0.35      0.41      0.37        64

avg / total       0.80      0.81      0.80      8652

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      7180
          1       0.81      0.49      0.61      1472

avg / total       0.89      0.89      0.88      8652

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      8652

aid_related
             precision    recall  f1-score   support

          0       0.80      0.80      0.80      5107
          1       0.71      0.70      0.71      3545

avg / total       0.76      0.76      0.76      8652

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [1]:
# testing a k-Nearest Neighbors classifier
from sklearn.neighbors import KNeighborsRegressor
kn_clf=KNeighborsRegressor()
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', kn_clf)
    ])

#train pipeline
pipeline.fit(X_train.values, Y_train.values)

#pipeline.fit(X_train.as_matrix(), Y_train.as_matrix())
y_pred = pipeline.predict(X_test)






NameError: name 'Pipeline' is not defined

### 9. Export your model as a pickle file

In [ ]:
m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.